In [1]:
# To run this file correctly, do the following
# indent : entire code inside to function
# delete : is_only_admin=0  and For Dev psycopg2 and dotnet env
# uncomment: For Production running on Python Enviroment
# uncomment : return and dtNow=datetime.now()

In [2]:
# def notify_monthly_pm_to_admin(is_only_admin):

In [3]:
import pandas as pd
from dotenv import dotenv_values

from datetime import datetime,date
from dateutil import relativedelta
import os


# Paramter & Config values

In [4]:
is_only_admin=0
is_only_admin=bool(is_only_admin)

temp_doc="temp_pm_notifcation"

# Retrive data from SMartDB Postgresql


In [5]:
# For Production running on Python Enviroment
# from django.db import connection
# def get_postgres_conn():
#   return connection


# For Dev
env_path=r'D:\PythonDev\Yit\SMartApp_ScriptDev\.env'
config = dotenv_values(dotenv_path=env_path)

import psycopg2
import psycopg2.extras as extras
def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error


In [6]:
def list_data(sql,params,connection):
 df=None   
 with connection.cursor() as cursor:
    
    if params is None:
       cursor.execute(sql)
    else:
       cursor.execute(sql,params)
    
    columns = [col[0] for col in cursor.description]
    dataList = [dict(zip(columns, row)) for row in cursor.fetchall()]
    df = pd.DataFrame(data=dataList) 
 return df 

# Get the next month 
* Set Window Sheduler to monthly run at the fist dsate of The month
* No matter what day you run this job , the program will  get only all PMs over the current month

In [7]:
#dtNow= datetime.strptime(datetime(2023,12,1,6,0).strftime('%Y-%m-%d'),'%Y-%m-%d')
dtNow=datetime.now()

dt = datetime.strptime(dtNow.strftime('%Y-%m-%d'),'%Y-%m-%d')
print(dt)

first_day_month = datetime(dt.year,dt.month,1)
first_day_next_month= dt + relativedelta.relativedelta(months=1, day=1)
print(first_day_month )   
print(first_day_next_month ) 


2023-11-30 00:00:00
2023-11-01 00:00:00
2023-12-01 00:00:00


# Create File Name and Path to attach to email

In [8]:
file_name=f"PM_{first_day_month.strftime('%b%y')}_{dtNow.strftime('%d%m%y%H%M')}.xlsx"
if is_only_admin==False:
   file_name=f"SM-{file_name}" 
else:
   file_name=f"NoneSM-{file_name}" 
file_path=f"{temp_doc}/{file_name}"
print(file_path)

temp_pm_notifcation/SM-PM_Nov23_3011231404.xlsx


# Retrive  and Transaform data

In [9]:
def do_something_df(df):
    df = df.fillna(value='')
    df = df.reset_index(drop=False)
    df.insert(0, "No", df["index"]+1,True)
    df=df.drop(columns=["index"])
    return df

In [10]:
# https://github.com/technqvi/SMartApp/blob/main/app/pm_doc_manager/pm_export.py

sql_pm=f"""
    select ac.company_full_name as "ชื่อลูกค้า",
    ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" ,
    ap.project_name as "ชื่อโครงการ",
    TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",
    TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
    pm.remark as  "งวดPM",
    ae.employee_name as "หัวหน้าทีม",
    (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Engineer"
    from app_preventivemaintenance pm
    left join app_project ap on ap.id = pm.project_id
    left join app_company ac on ac.id = ap.company_id
    left join app_employee ae on ae.id =pm.team_lead_id
    where  pm.planned_date>='{first_day_month}' and pm.planned_date<'{first_day_next_month}'
    and ac.is_managed_by_admin = {is_only_admin}
    order by  ac.company_full_name,ap.enq_id,pm.remark
    """
# print(sql_pm)

In [11]:
dfPM=list_data(sql_pm,None,get_postgres_conn())
dfPM=do_something_df(dfPM)
dfPM.info()
dfPM.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   No                 73 non-null     int64 
 1   ชื่อลูกค้า         73 non-null     object
 2   เลขที่สัญญา        73 non-null     object
 3   ENQ                73 non-null     object
 4   ชื่อโครงการ        73 non-null     object
 5   แผนจะทำPM          73 non-null     object
 6   วันสุดท้ายที่ทำPM  73 non-null     object
 7   งวดPM              73 non-null     object
 8   หัวหน้าทีม         73 non-null     object
 9   Engineer           73 non-null     object
dtypes: int64(1), object(9)
memory usage: 5.8+ KB


,No,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,แผนจะทำPM,วันสุดท้ายที่ทำPM,งวดPM,หัวหน้าทีม,Engineer
0,1,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,Nov 2023,30 Nov 2023,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala
1,2,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,Nov 2023,30 Nov 2023,1/2 : HPE CSOC (Physical),Boonlert Kawta,Paradon Srimode
2,3,Bangkok Bank Public Company Limited,,ENQ/65-412,BBL-โครงการ MA 2 ปี สำหรับ NetApp AFF 800,Nov 2023,30 Nov 2023,5/8,Boonlert Kawta,Sitthikorn Prakorngjitman
3,4,Bangkok Bank Public Company Limited,,ENQ/66-999,BBL Project Test 01,Nov 2023,30 Nov 2023,1/4 - NetApp- DC,Boonlert Kawta,Sitthikorn Prakorngjitman
4,5,Bangkok Bank Public Company Limited,,ENQ/66-999,BBL Project Test 01,Nov 2023,30 Nov 2023,1/4 - NetApp - DR,Boonlert Kawta,Sitthikorn Prakorngjitman


In [12]:
sql_item = f"""
 select  ac.company_full_name as "ชื่อลูกค้า",
 ap.contract_no as "เลขที่สัญญา",ap.enq_id as "ENQ" , ap.project_name as "ชื่อโครงการ",

             ai.serial_number as "Serial",
             (select  productype_name from app_product_type where id=ai.product_type_id ) as "ProudctType",
             (select  brand_name from app_brand where id=ai.brand_id ) as "Brand",
              (select  model_name from app_model where id=ai.model_id ) as "Model",

             TO_CHAR(pm.planned_date,'Mon YYYY') as "แผนจะทำPM",TO_CHAR(pm.ended_pm_date,'DD Mon YYYY') as "วันสุดท้ายที่ทำPM",
             pm.remark as  "งวดPM",ae.employee_name as "หัวหน้าทีม",
             (select emp.employee_name emp from app_employee emp where emp.id=pm.engineer_id ) as "Planed Engineer",

           (select employee_name from app_employee eng_pm  where eng_pm.id=pm_item.pm_engineer_id ) as "Operation Engineer",
           TO_CHAR(pm_item.actual_date,'DD Mon YYYY') as "ActualDate",pm_item.call_number as "Call Number",

            (select employee_name from app_employee eng_doc  where eng_doc.id=pm_item.document_engineer_id ) as "Doc Engineer",
           TO_CHAR(pm_item.document_date,'DD Mon YYYY') as "DocumentDate",pm_item.pm_document_number as "Doc Number",
           pm_item.remark as "Remark"

from app_pm_inventory as pm_item
left join app_inventory ai on ai.id = pm_item.inventory_id
-- inner join app_product_type  product_type on ai.product_type_id = product_type.id
left join app_preventivemaintenance pm on pm.id = pm_item.pm_master_id
left join app_project ap on ap.id = pm.project_id
left join app_company ac on ac.id = ap.company_id
left join app_employee ae on ae.id =pm.team_lead_id

where pm_item.is_pm=True and ac.is_managed_by_admin = {is_only_admin}
and  pm.planned_date>='{first_day_month}' and pm.planned_date<'{first_day_next_month}'

order by  ac.company_full_name,ap.enq_id,pm.remark

    """

# print(sql_item)

In [13]:
dfItem=list_data(sql_item,None,get_postgres_conn())
dfItem=do_something_df(dfItem)
dfItem.info()
dfItem.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   No                  932 non-null    int64 
 1   ชื่อลูกค้า          932 non-null    object
 2   เลขที่สัญญา         932 non-null    object
 3   ENQ                 932 non-null    object
 4   ชื่อโครงการ         932 non-null    object
 5   Serial              932 non-null    object
 6   ProudctType         932 non-null    object
 7   Brand               932 non-null    object
 8   Model               932 non-null    object
 9   แผนจะทำPM           932 non-null    object
 10  วันสุดท้ายที่ทำPM   932 non-null    object
 11  งวดPM               932 non-null    object
 12  หัวหน้าทีม          932 non-null    object
 13  Planed Engineer     932 non-null    object
 14  Operation Engineer  932 non-null    object
 15  ActualDate          932 non-null    object
 16  Call Number         932 no

,No,ชื่อลูกค้า,เลขที่สัญญา,ENQ,ชื่อโครงการ,Serial,ProudctType,Brand,Model,แผนจะทำPM,...,งวดPM,หัวหน้าทีม,Planed Engineer,Operation Engineer,ActualDate,Call Number,Doc Engineer,DocumentDate,Doc Number,Remark
0,1,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,SGH035S31B,Server,HPE,ProLiant DL560 Gen10,Nov 2023,...,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala,,,,,,,
1,2,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,SGH035S31L,Server,HPE,ProLiant DL560 Gen10,Nov 2023,...,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala,,,,,,,
2,3,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,SGH932V6SQ,Server,HPE,ProLiant DL560 Gen10,Nov 2023,...,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala,,,,,,,
3,4,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,SGH932V6SV,Server,HPE,ProLiant DL560 Gen10,Nov 2023,...,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala,,,,,,,
4,5,Advanced Info Services Public Company Limited,,ENQ/66-487,Preventive Maintenance HPE 12 Servers AIS CSOC...,SGH932V6SZ,Server,HPE,ProLiant DL560 Gen10,Nov 2023,...,1/2 : HPE CSOC (Logical),Yutthana Klinmala,Tura Klinmala,,,,,,,


# Excel file

In [14]:
with pd.ExcelWriter(file_path) as writer:
    dfPM.to_excel(writer, sheet_name="PM-Plan",index=False)
    dfItem.to_excel(writer, sheet_name="PM-Item",index=False)
    print(f"Exported {file_name} file for email successfully.")

Exported SM-PM_Nov23_3011231404.xlsx file for email successfully.


# Email Office 365

# Delete attach files : if sent email with excel file completely

In [15]:

os.remove(f"{file_path}")

In [ ]:
# return True